In [34]:
import requests
import json
import warnings
import pandas as pd
from datetime import datetime, timedelta, date
from pprint import pprint

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# OBTENCIÓN DE DATOS DE INTERCAMBIO 

In [3]:
lang = input('¿En español (es) o en inglés (en)?')
restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [4]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [5]:
def intercambio_datos(lang, restaDia, input_año):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
   


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                

                fecha =  fecha.strftime("%d/%m/%Y")

                
                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
               


    df_intercambio = pd.DataFrame()
    df_intercambio['nombre']               = lista_nombres
    df_intercambio['tipo de intercambio']      = lista_tipos
    df_intercambio['Valores']              = lista_valores
    df_intercambio["Porcentaje"]           = lista_porcentajes
    df_intercambio["Fecha actualización"]  = lista_fechas
             


    return df_intercambio 

df_intercambio = intercambio_datos(lang, restaDia, input_año)

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-11122.5,0.202408,11/09/2023
1,Francia,Exportación,-18856.9,0.299353,12/09/2023
2,Francia,Exportación,-17628.5,0.367195,13/09/2023
3,Francia,Exportación,-7880.3,0.143426,14/09/2023
4,Francia,Exportación,-2572.1,0.043121,15/09/2023
...,...,...,...,...,...
325,Andorra,saldo,-399.0,1.000000,07/10/2023
326,Andorra,saldo,-384.0,1.000000,08/10/2023
327,Andorra,saldo,-439.0,1.000000,09/10/2023
328,Andorra,saldo,-865.0,1.000000,10/10/2023


In [7]:
df_intercambio.to_csv('intercambio_electrico.csv')

In [6]:
fig = px.bar(df_intercambio, x = 'nombre', y = 'Valores', color = 'tipo de intercambio',
             title= 'Valor del intercambio por País y Tipo',
             labels= {'Valores' : 'Valor de intercambio', 'nombre' : 'nombre'},
             barmode= 'group')

fig.show()

In [7]:
fig = px.scatter(df_intercambio, x = 'Valores', y = 'Porcentaje', color = 'nombre',
                 size= 'Porcentaje', hover_name= 'tipo de intercambio',            
                 title = 'Relación entre el valor del intercambio y el porcentaje de cambio',
                 size_max= 10)

fig.show()

In [47]:
fig = px.line(df_intercambio, x = 'Fecha actualización', y = 'Valores', color= 'nombre',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por País y Tipo',
              markers= True)

fig.show()

In [46]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Francia'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio Francia',
              markers= True)

fig.show()

In [45]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Portugal'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Portugal',
              markers= True)

fig.show()

In [44]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Marruecos'], x = 'Fecha actualización', y = 'Valores', color= 'tipo de intercambio',
              line_group='tipo de intercambio',
              title= 'Evolucion de los valores de intercambio por Marruecos',
              markers= True)

fig.show()

In [42]:
fig = px.line(df_intercambio[df_intercambio['nombre'] == 'Andorra'], 
              x='Fecha actualización', y='Valores', color='tipo de intercambio',
              line_group='tipo de intercambio',
              title='Evolución de los valores de intercambio por Andorra',
              markers=True,  
              line_dash='tipo de intercambio')  

fig.show()
